# Lab 5

In [555]:
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
import torch 
from torch import nn

* Write ResNet architecture by pytorch.

I will try implement [ResNet-34](https://arxiv.org/pdf/1512.03385).

1. Implement resnet block

In [556]:
class ResBlock(nn.Module):
    """one block of resnet_34 model

    Args:
        in_out_channels (int): input and output channels
        n (int): expansion output channel
    """
    def __init__(self, in_out_channels, n=1):
        super().__init__()
        layers = list()  

        layers.append(nn.Conv2d(in_out_channels, in_out_channels, kernel_size=3, stride=1, padding=1))
        layers.append(nn.BatchNorm2d(in_out_channels))
        layers.append(nn.ReLU())
        
        layers.append(nn.Conv2d(in_out_channels, in_out_channels*n, kernel_size=3, stride=1, padding=1))
        layers.append(nn.BatchNorm2d(in_out_channels*n))
        
        self.layers = nn.Sequential(*layers)
        self.activastion = nn.ReLU()
        
        # if dimension is up
        if n > 1:
            w = list()
            w.append(nn.Conv2d(in_out_channels, in_out_channels*n, kernel_size=3, stride=1, padding=1))
            w.append(nn.BatchNorm2d(in_out_channels*n))
            self.weights = nn.Sequential(*w)
        else:
            self.weights = lambda x: x
        
    def forward(self, x):
        return self.activastion( self.layers(x) + self.weights(x) )

2. Implement resnet layers

In [557]:
class convi_x(nn.Module):
    def __init__(self, in_out_channels, blocks, is_last=False):
        """conv2_x, conv3_x, conv4_x, conv5_x layers in one class

        Args:
            in_out_channels (int): input and output channels
            blocks (int): num of blocks
            is_last (bool): var for checking is it conv5_x
        """
        super().__init__()
        layers = list()  
        
        if is_last:
            for i in range(blocks):           
                layers.append(ResBlock(in_out_channels))
        else:
            for i in range(blocks-1):           
                layers.append(ResBlock(in_out_channels))
            layers.append(ResBlock( in_out_channels, n = 2 ))
        
            
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

3. Implement resnet model

In [558]:
class ResNet_34(nn.Module):
    def __init__(self, output_channels):
        super().__init__()
        # Convolutions
        blocks = [(64, 2, 0) , (128, 3, 0), (256, 6, 0), (512, 3, 1)] # in_out_channels, num of blocks and bool var for conv5_x
        layers = list()
        
        layers.append(nn.Conv2d(3, 64, kernel_size=7, stride=2)) # conv1
        
        layers.append(nn.MaxPool2d(3, 2))
        
        for block in blocks:
            layers.append(convi_x(block[0], block[1], block[2]))
            
        
        layers.append(nn.AdaptiveAvgPool2d(1))    
        
        self.layers = nn.Sequential(*layers)
        
        # FC layers + Softmax activation
        self.fcl = nn.Linear(512, output_channels)
        self.activastion = nn.Softmax()
    
    def forward(self, x):
        conv_out = self.layers(x)
        conv_out = torch.flatten(conv_out, start_dim=1)
        return self.activastion(self.fcl(conv_out))

In [559]:
res = ResNet_34(1)
x = torch.randn(1, 3, 64, 64)

res.forward(x)

c:\Users\Омар\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[1.]], grad_fn=<SoftmaxBackward0>)